In [ ]:
%%capture
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
import altair as alt
import branca
import branca.colormap as cm
import calitp_data_analysis.magics
import geopandas as gpd
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html

from shared_utils import webmap_utils
from update_vars import GTFS_DATA_DICT, file_name

from omegaconf import OmegaConf
readable_dict = OmegaConf.load("new_readable.yml")

import google.auth
credentials, project = google.auth.default()
import _new_operator_report_utils as utils

In [ ]:
from calitp_data_analysis.gcs_pandas import GCSPandas
from functools import cache

In [ ]:
@cache
def gcs_pandas():
    return GCSPandas()

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
def formatted(number):
    return "{:,}".format(number)

In [ ]:
# analysis_name = 'City of Santa Monica'

In [ ]:
%%capture_parameters
analysis_name

In [ ]:
# Set drop down menu to be on the upper right for the charts
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

In [ ]:
GCS_PATH = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/"

In [ ]:
analysis_name_edited = analysis_name.replace(" ","_").lower()
operator_webmap_file = f"{analysis_name_edited}_routes"

In [ ]:
fct_monthly_routes_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.route_map}_{file_name}.parquet"

In [ ]:
schedule_rt_route_direction_summary_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.schedule_rt_route_direction}_{file_name}.parquet"

In [ ]:
operator_hourly_summary_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.hourly_day_type_summary}_{file_name}.parquet"

In [ ]:
ntd_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.ntd_profile}_{file_name}.parquet"

In [ ]:
operator_summary_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary}_{file_name}.parquet"

In [ ]:
fct_monthly_route_df = gpd.read_parquet(fct_monthly_routes_url,
                                        filters=[[("Analysis Name", "==", analysis_name)]],
                                        storage_options={"token": credentials.token}).reset_index()

In [ ]:
fct_monthly_route_df['Number'] = fct_monthly_route_df.index

In [ ]:
operator_hourly_summary_df = gcs_pandas().read_parquet(operator_hourly_summary_url,
                                             filters=[[("Analysis Name", "==", analysis_name),
                                                      ("Departure Hour", "<=", 24)]])

In [ ]:
schedule_rt_route_direction_summary_df = gcs_pandas().read_parquet(schedule_rt_route_direction_summary_url,
                                                         filters=[[("Analysis Name", "==", analysis_name)]])

In [ ]:
ntd_profile_df =  gcs_pandas().read_parquet(ntd_url, filters=[[("analysis_name", "==", analysis_name)]])

In [ ]:
operator_df = gcs_pandas().read_parquet(
    operator_summary_url,
    filters=[
        ("Day Type", "==", "Weekday"),
        ("Analysis Name", "==", analysis_name)],
).drop_duplicates(subset = ["Analysis Name", "Date"])

# {analysis_name}

## Operator Overview

In [ ]:
try:
    service_area = formatted(int(ntd_profile_df.service_area_sq_miles.values[0]))
    service_pop = formatted(int(ntd_profile_df.service_area_pop.values[0]))
except:
    pass

In [ ]:
try:
    display(
        Markdown(
            f"""{analysis_name} is headquartered in <b>{ntd_profile_df.hq_county.values[0]}</b> County in the Urbanized Area of <b>{ntd_profile_df.primary_uza_name.values[0]}</b>.<br>
            This operator provides <b>{service_area}</b> square miles of public transit service, which has a service population of <b>{service_pop}</b>.<br>
            This organization is a {ntd_profile_df.reporter_type.values[0]}.<br>
            <b>Data Source</b>: <a href="https://www.transit.dot.gov/ntd/data-product/2022-annual-database-agency-information">National Transit Database</a> Annual Agency Information.
            """
        )
    )
except:
    pass

## Route Typologies

In [ ]:
try:
    n_routes = formatted(schedule_rt_route_direction_summary_df["Route"].nunique())
    display(
        Markdown(
            f"""{analysis_name} runs <b>{n_routes}</b> unique routes. Below is the breakdown of the routes. Routes can belong to one or more categories.<p>
            Route categories are determined using a approach that looks at GTFS trips data
        alongside National Association of City Transportation Officials (NACTO)'s
        <a href="https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-route-types/">Transit Route Types</a> 
        and <a href= "https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-frequency-volume/">Frequency and Volume</a>
        guides. Please see the <a href="https://github.com/cal-itp/data-analyses/blob/main/gtfs_digest/methodology.md">methodology docs</a> for more details on this approach.
        """
        )
    )
except:
    display(Markdown(f"""{analysis_name} doesn't have an operator profile."""))

In [ ]:
try:
    display(utils.create_route_typology(schedule_rt_route_direction_summary_df))
except:
    display(Markdown(f"""{analysis_name} doesn't have information on route typologies."""))

## Route Length

In [ ]:
try:
    display(utils.create_route_lengths(fct_monthly_route_df))
except:
    display(Markdown(f"""{analysis_name} doesn't have information on route lengths."""))

## Service Area

In [ ]:
color_map = cm.linear.Spectral_11.scale(0, len(fct_monthly_route_df))

In [ ]:
try:
    operator_map = webmap_utils.set_state_export(
    fct_monthly_route_df.drop_duplicates(subset = ["Route Name"])[["Route Name","Geometry", "Number"]],
    subfolder = "operator_digest/",
    filename=operator_webmap_file,
    map_title=f"Transit Routes for {analysis_name}",
    cmap=color_map,
    color_col="Number",
    overwrite = True
    )
    webmap_utils.render_spa_link(operator_map["spa_link"], text=f"Routes for {analysis_name}")
    webmap_utils.display_spa_map(operator_map["spa_link"])
except:
    display(Markdown(f"""{analysis_name} doesn't have an route geographies."""))

## Service Hours 

In [ ]:
try:
    display(utils.create_hourly_summary(operator_hourly_summary_df, "Weekday"))
    display(utils.create_hourly_summary(operator_hourly_summary_df, "Saturday"))
    display(utils.create_hourly_summary(operator_hourly_summary_df, "Sunday"))
except: 
    display(Markdown(f"""{analysis_name} doesn't have service hours information."""))

## Trip Update Overview

In [ ]:
try:
    display(utils.create_tu_minute(operator_df))
    display(utils.create_tu_pct(operator_df))
except:
    display(Markdown(f"""{analysis_name} doesn't have trip update information."""))

## Vehicle Positions Overview

In [ ]:
try:
    display(utils.create_vp_minute(operator_df))
    display(utils.create_vp_pct(operator_df))
except:
    display(Markdown(f"""{analysis_name} doesn't have vehicle positions information."""))

## Detailed Route Overview

In [ ]:
try:
    display(utils.create_scheduled_minutes(schedule_rt_route_direction_summary_df))
    display(utils.create_frequency(schedule_rt_route_direction_summary_df))
    display(utils.create_text_graph(schedule_rt_route_direction_summary_df))
except:
    display(Markdown(f"""{analysis_name} doesn't have detailed route information."""))